# Notebook Objective:
- Exctracting ocr_bbox and index info from DOCVQA raw data folder

In [1]:
!pip install -q transformers
!pip install transformers[sentencepiece]
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.4/492.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import os
import json
from PIL import Image
from datasets import Dataset
import inspect
import transformers
import string

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
## main training json path
train_path_main = '/content/drive/MyDrive/docVqa_dataset/train/train_v1.0.json'

In [5]:
# Open the JSON file
with open(train_path_main, 'r') as file:

  # Load the contents of the file into a dictionary
  data_train = json.load(file)

In [6]:
data_train['data'][0]         ## sample data

{'questionId': 337,
 'question': 'what is the date mentioned in this letter?',
 'image': 'documents/xnbl0037_1.png',
 'docId': 279,
 'ucsf_document_id': 'xnbl0037',
 'ucsf_document_page_no': '1',
 'answers': ['1/8/93'],
 'data_split': 'train'}

In [7]:
## Convert 'data' list into dataframe by passing into the dataframe
df_train = pd.DataFrame(data_train['data'])

In [8]:
## Making subset of the training dataframe
df_sub = df_train.iloc[:50]

In [9]:
base_train_path = '/content/drive/MyDrive/docVqa_dataset/train'

In [10]:
# Define a function to join the base path with the image path
def join_paths(image_path):
    return os.path.join(base_train_path, image_path)

# Apply the function to create a new column with the full image path
df_sub['full_path_image'] = df_sub['image'].apply(join_paths)

<ipython-input-10-0e329387f7a0>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub['full_path_image'] = df_sub['image'].apply(join_paths)


In [11]:
df_sub.head()

,questionId,question,image,docId,ucsf_document_id,ucsf_document_page_no,answers,data_split,full_path_image
0,337,what is the date mentioned in this letter?,documents/xnbl0037_1.png,279,xnbl0037,1,[1/8/93],train,/content/drive/MyDrive/docVqa_dataset/train/do...
1,338,what is the contact person name mentioned in l...,documents/xnbl0037_1.png,279,xnbl0037,1,"[P. Carter, p. carter]",train,/content/drive/MyDrive/docVqa_dataset/train/do...
2,339,Which corporation's letterhead is this?,documents/mxcj0037_1.png,280,mxcj0037,1,[Brown & Williamson Tobacco Corporation],train,/content/drive/MyDrive/docVqa_dataset/train/do...
3,340,Who is in cc in this letter?,documents/mxcj0037_1.png,280,mxcj0037,1,[T.F. Riehl],train,/content/drive/MyDrive/docVqa_dataset/train/do...
4,341,what is the subject of this letter?,documents/mxcj0037_1.png,280,mxcj0037,1,[Review of existing Brainstorming Ideas/483],train,/content/drive/MyDrive/docVqa_dataset/train/do...


In [12]:
# ## Checking list of dirs and files in the main train folder
# dir_list = os.listdir(base_train_path)    ## train dir contains two sub dir and one json file
# dir_list, dir_list[1]

In [13]:
# ocr_base_dir = os.path.join(base_train_path, dir_list[1])
# ocr_base_dir

In [14]:
# df_train['ocr_path'] = df_train.apply(lambda row: ocr_base_dir + '/' + row['ucsf_document_id'] + '_' + row['ucsf_document_page_no'] + '.json', axis=1)

In [15]:
# df_train['ocr_path'][0]

In [16]:
# df_train['ocr_path'][1]

In [17]:
# df_train.head()

In [18]:
def extract_ocr_info(df, base_train_dir):

  dir_list = os.listdir(base_train_path)
  ocr_base_dir = os.path.join(base_train_path, dir_list[1])

  # Create empty lists for bounding boxes and word lists
  bbox_list = []
  word_list = []

  # Iterate over each row in the DataFrame
  for _, row in df.iterrows():

    # Construct the ocr_path for the row
    ocr_path = os.path.join(ocr_base_dir, row['ucsf_document_id'] + '_' + row['ucsf_document_page_no'] + '.json')

    # Open the json file and load the data
    with open(ocr_path, 'r') as f:
      ocr_json = json.load(f)

    # Extract the bounding boxes and word lists from the json data
    bbox_row = []
    word_row = []
    for i in ocr_json['recognitionResults'][0]['lines']:
      for j in i['words']:
        bbox_row.append(j['boundingBox'])
        word_row.append(j['text'])

    # Append the bbox and word lists to the global lists
    bbox_list.append(bbox_row)
    word_list.append(word_row)
  # Create new columns in the DataFrame to store the bbox and words lists
  df['bounding_boxes'] = bbox_list
  df['word_list'] = word_list

  df.drop(['questionId', 'image','ucsf_document_id', 'ucsf_document_page_no'], axis=1, inplace=True)

  # Return the updated DataFrame
  return df


In [19]:
df_sub = extract_ocr_info(df_sub, base_train_path)

<ipython-input-18-e3b5e856659a>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['bounding_boxes'] = bbox_list
<ipython-input-18-e3b5e856659a>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['word_list'] = word_list
<ipython-input-18-e3b5e856659a>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['question

In [20]:
df_sub.head()

,question,docId,answers,data_split,full_path_image,bounding_boxes,word_list
0,what is the date mentioned in this letter?,279,[1/8/93],train,/content/drive/MyDrive/docVqa_dataset/train/do...,"[[586, 30, 993, 36, 994, 119, 584, 131], [1080...","[Confidential, .., .., RJRT, PR, APPROVAL, DAT..."
1,what is the contact person name mentioned in l...,279,"[P. Carter, p. carter]",train,/content/drive/MyDrive/docVqa_dataset/train/do...,"[[586, 30, 993, 36, 994, 119, 584, 131], [1080...","[Confidential, .., .., RJRT, PR, APPROVAL, DAT..."
2,Which corporation's letterhead is this?,280,[Brown & Williamson Tobacco Corporation],train,/content/drive/MyDrive/docVqa_dataset/train/do...,"[[733, 196, 789, 193, 791, 218, 734, 219], [47...","[B&W, BROWN, &, WILLIAMSON, TOBACCO, CORPORATI..."
3,Who is in cc in this letter?,280,[T.F. Riehl],train,/content/drive/MyDrive/docVqa_dataset/train/do...,"[[733, 196, 789, 193, 791, 218, 734, 219], [47...","[B&W, BROWN, &, WILLIAMSON, TOBACCO, CORPORATI..."
4,what is the subject of this letter?,280,[Review of existing Brainstorming Ideas/483],train,/content/drive/MyDrive/docVqa_dataset/train/do...,"[[733, 196, 789, 193, 791, 218, 734, 219], [47...","[B&W, BROWN, &, WILLIAMSON, TOBACCO, CORPORATI..."


In [21]:
# # Save the DataFrame as a CSV file
# df_sub.to_csv('data_sub.csv', index=False)

## Converting dataframe sub to huggingface dataset format

In [22]:
## Convert dataframe into dataset object
hfdataset_sub = Dataset.from_pandas(df_sub)

In [23]:
hfdataset_sub

Dataset({
    features: ['question', 'docId', 'answers', 'data_split', 'full_path_image', 'bounding_boxes', 'word_list'],
    num_rows: 50
})

In [24]:
## Custom function to identify indexing and collect bbox values

def get_info(hfexample):

  words = hfexample["word_list"]
  bbox = hfexample["bounding_boxes"]
  ans = hfexample["answers"]

  ###converting the words list to a single string
  sentence = " ".join(words)

  ###Removing all the punc from the string
  for punctuation in string.punctuation:
    sentence = sentence.replace(punctuation, '')

  ###Removing all the punc from the output string
  ans_sentence = ""
  for element in ans:
    for punctuation in string.punctuation:
      element = element.replace(punctuation, '')
    ans_sentence = ans_sentence + element + " "

  ###converting the strings to list
  sentence = list(sentence.split(" "))
  answer = list(ans_sentence.split(" "))

  ###Finding the index of the output words in the main string's words.
  answer_index = []
  for word in answer:
    if word:
      try:
        index = sentence.index(word)
        answer_index.append(index)
      except ValueError:
        pass

  ###Finding the bb_values of the output words.
  bb_values = []
  for index in answer_index:
    bb_values.append(bbox[index])

  # Creating the info dictionary
  info = {
      "actual_ans": [word for word in ans if word],
      "answer_index": answer_index,
      "bbox_values": bb_values
  }

  # Add the info dictionary to the example
  hfexample["detail_info"] = info

  return hfexample


In [25]:
# Apply the custom function to the dataset using map()
hfdataset_sub_info = hfdataset_sub.map(get_info)

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [26]:
# Print the modified dataset
hfdataset_sub_info

Dataset({
    features: ['question', 'docId', 'answers', 'data_split', 'full_path_image', 'bounding_boxes', 'word_list', 'detail_info'],
    num_rows: 50
})

#### Sample Data testing

In [27]:
def sampleDetailschecking(k):
  print(hfdataset_sub_info[k])
  print('*****************************************')
  print(hfdataset_sub_info[k]['question'])
  print('*****************************************')
  print(hfdataset_sub_info[k]['answers'])
  print('*****************************************')
  print(hfdataset_sub_info[k]['bounding_boxes'])
  print('*****************************************')
  print(hfdataset_sub_info[k]['word_list'])
  print('*****************************************')
  print(hfdataset_sub_info[k]['detail_info'])


In [28]:
sampleDetailschecking(49)

{'question': 'According to the listed requirements , what must be the age group of female smokers?', 'docId': 319, 'answers': ['between the ages of 18 and 55'], 'data_split': 'train', 'full_path_image': '/content/drive/MyDrive/docVqa_dataset/train/documents/fxxj0037_2.png', 'bounding_boxes': [[214, 177, 364, 174, 363, 209, 216, 209], [217, 260, 274, 261, 275, 293, 219, 292], [281, 261, 354, 261, 355, 293, 282, 293], [361, 261, 449, 262, 450, 294, 362, 293], [456, 262, 519, 263, 519, 294, 456, 294], [525, 263, 658, 264, 658, 295, 526, 294], [664, 264, 700, 264, 700, 296, 665, 295], [707, 264, 774, 264, 774, 296, 707, 296], [780, 264, 823, 265, 822, 296, 780, 296], [829, 265, 854, 265, 854, 297, 829, 296], [861, 265, 920, 265, 919, 297, 860, 297], [926, 265, 1021, 265, 1020, 298, 925, 297], [1027, 265, 1147, 266, 1146, 298, 1026, 298], [1154, 266, 1181, 266, 1180, 299, 1153, 298], [1188, 266, 1291, 266, 1289, 299, 1186, 299], [216, 301, 418, 302, 418, 334, 216, 333], [424, 302, 452, 302,